# Week 2: Image Processing Basics — Lab Practice

**Topics:** Pixels, Point Processing & Histograms

This notebook accompanies the Week 2 lecture slides. We will load images as NumPy arrays, perform point processing operations, compute histograms, and explore color spaces — all hands-on.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb

%matplotlib inline

### Environment setup

This notebook works both **locally** and on **Google Colab**.
- **Local**: images are loaded from the repository's `images/` folder.
- **Colab**: images are automatically downloaded from GitHub on first run.

In [ ]:
import os, urllib.request

# Detect Google Colab
IN_COLAB = "google.colab" in str(get_ipython()) if hasattr(__builtins__, "__IPYTHON__") else False

# Image paths
REPO_URL = "https://raw.githubusercontent.com/HyeongminLEE/image-processing-tutorial/main"
IMAGE_DIR = "images"
IMAGE_NAME = "parrots_square.jpg"

if IN_COLAB:
    os.makedirs(IMAGE_DIR, exist_ok=True)
    url = f"{REPO_URL}/{IMAGE_DIR}/{IMAGE_NAME}"
    dest = os.path.join(IMAGE_DIR, IMAGE_NAME)
    if not os.path.exists(dest):
        print(f"Downloading {IMAGE_NAME} ...")
        urllib.request.urlretrieve(url, dest)
        print("Done.")
    IMAGE_PATH = dest
else:
    IMAGE_PATH = os.path.join("..", IMAGE_DIR, IMAGE_NAME)

GRAY_PATH = "parrots_gray.jpg"

print(f"Image path: {IMAGE_PATH}")
print(f"Running on: {'Google Colab' if IN_COLAB else 'Local'}")

### Display helpers

We define two utility functions used throughout this notebook so that visualization code stays minimal.

In [ ]:
def show_images(*imgs, titles=None, scale=4):
    """Display images in a single row.

    Grayscale (2-D uint8) arrays automatically use a gray colormap.
    *titles* is an optional list of strings, one per image.
    """
    n = len(imgs)
    fig, axes = plt.subplots(1, n, figsize=(scale * n, scale))
    if n == 1:
        axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if img.ndim == 2:
            ax.imshow(img, cmap="gray", vmin=0, vmax=255)
        else:
            ax.imshow(img)
        if titles:
            ax.set_title(titles[i])
        ax.axis("off")
    plt.tight_layout()
    plt.show()


def show_with_hist(*imgs, titles=None, scale=4):
    """Display grayscale images (top row) with their histograms (bottom row)."""
    n = len(imgs)
    fig, axes = plt.subplots(2, n, figsize=(scale * n, scale * 1.5))
    if n == 1:
        axes = axes.reshape(2, 1)
    for i, img in enumerate(imgs):
        axes[0, i].imshow(img, cmap="gray", vmin=0, vmax=255)
        if titles:
            axes[0, i].set_title(titles[i])
        axes[0, i].axis("off")
        h, _ = np.histogram(img.ravel(), 256, [0, 256])
        axes[1, i].fill_between(np.arange(256), h, alpha=0.7)
        axes[1, i].set_xlim([0, 255])
    plt.tight_layout()
    plt.show()

---
## 0. NumPy Crash Course

Images in Python are just **NumPy arrays**. Before touching any pixels, let's get comfortable with the array operations we'll use throughout this course: creating, slicing, reshaping, flipping, and combining arrays.

### Creating arrays

In [ ]:
# From a Python list
a = np.array([1, 2, 3, 4, 5])
print("a       =", a)
print("a.shape =", a.shape)      # (5,)  — 1-D, 5 elements
print("a.ndim  =", a.ndim)       # 1

In [ ]:
# 2-D array (matrix)
b = np.array([[1, 2, 3],
              [4, 5, 6]])
print("b.shape =", b.shape)         # (2, 3) — 2 rows, 3 columns
print(b)

In [ ]:
# Handy constructors
print("zeros:\n", np.zeros((2, 3)))                  # all 0s
print("ones:\n", np.ones((2, 3)))                    # all 1s
print("arange  :", np.arange(0, 10, 2))              # [0, 2, 4, 6, 8]  (start, stop, step)
print("linspace:", np.linspace(0, 1, 5))             # 5 evenly spaced values in [0, 1]

# Create arrays that match the shape of an existing array
ref = np.array([[1, 2], [3, 4]])
print("zeros_like:\n", np.zeros_like(ref))           # same shape, all 0
print("ones_like:\n", np.ones_like(ref))             # same shape, all 1
print("copy:\n", ref.copy())                         # independent deep copy

### Indexing & slicing

Works like Python lists, but extended to multiple dimensions with **comma-separated indices**: `array[row, col]`.

In [ ]:
m = np.arange(12).reshape(3, 4)   # 3×4 matrix  [0..11]
print(m)
print()

print("m[0, 2]      =", m[0, 2])        # single element: row 0, col 2
print("m[1]         =", m[1])            # entire row 1
print("m[:, 0]      =", m[:, 0])         # entire column 0  (: means 'all rows')
print(f"m[0:2, 1:3]  =\n{m[0:2, 1:3]}")  # sub-matrix: rows 0-1, cols 1-2

In [ ]:
# Negative indexing & reversing
print("last row      =", m[-1])           # last row
print(f"rows reversed =\n{m[::-1]}")       # reverse row order → vertical flip
print(f"cols reversed =\n{m[:, ::-1]}")    # reverse column order → horizontal flip

### Element-wise arithmetic

Operators `+`, `-`, `*`, `/`, `**` work **element-by-element**. A scalar is automatically **broadcast** to match the array shape.

In [ ]:
v = np.array([10, 20, 30])

print("v + 5   =", v + 5)           # scalar broadcast: [15, 25, 35]
print("v * 2   =", v * 2)           # [20, 40, 60]
print("v ** 2  =", v ** 2)          # [100, 400, 900]
print("v > 15  =", v > 15)          # boolean array: [False, True, True]

In [ ]:
# Two arrays of the same shape → element-wise
w = np.array([1, 2, 3])
print("v + w =", v + w)             # [11, 22, 33]
print("v * w =", v * w)             # [10, 40, 90]

### Reshape, ravel, flatten

These change the **logical shape** of an array without changing its data.

In [ ]:
a = np.arange(12)
print("1-D      =", a)              # [0, 1, ..., 11]   shape (12,)

b = a.reshape(3, 4)                  # → shape (3, 4)
print(f"reshaped =\n{b}")

print("ravel    =", b.ravel())       # back to 1-D  (view — shares memory)
print("flatten  =", b.flatten())     # back to 1-D  (always a new copy)

### Adding / removing dimensions: expand_dims & squeeze

These are important when working with images:
- A grayscale image has shape `(H, W)` — but some functions expect `(H, W, 1)`.
- `np.expand_dims(arr, axis)` inserts a **new axis of size 1** at the given position.
- `np.squeeze(arr)` removes **all axes of size 1**.

In [ ]:
a = np.zeros((4, 5))               # shape (4, 5)
print("original shape   :", a.shape)

# expand_dims — add a dimension
b = np.expand_dims(a, axis=2)       # (4, 5) → (4, 5, 1)
print("expand axis=2    :", b.shape)

c = np.expand_dims(a, axis=0)       # (4, 5) → (1, 4, 5)
print("expand axis=0    :", c.shape)

# Shorthand: index with None (same as np.newaxis)
d = a[:, :, None]                   # (4, 5) → (4, 5, 1)
print("[:, :, None]     :", d.shape)

e = a[None, :, :]                   # (4, 5) → (1, 4, 5)
print("[None, :, :]     :", e.shape)

In [ ]:
# squeeze — remove all size-1 dimensions
f = np.zeros((1, 4, 5, 1))
print("before squeeze:", f.shape)    # (1, 4, 5, 1)
print("after  squeeze:", np.squeeze(f).shape)  # (4, 5)

### Concatenate & stack

- `np.concatenate` joins arrays along an **existing** axis.
- `np.stack` joins arrays along a **new** axis.

In [ ]:
p = np.ones((2, 3))
q = np.zeros((2, 3))

# concatenate along axis 0 (stack rows)
r = np.concatenate([p, q], axis=0)
print("concat axis=0 :", r.shape)   # (4, 3)
print(r)
print()

# concatenate along axis 1 (append columns)
s = np.concatenate([p, q], axis=1)
print("concat axis=1 :", s.shape)   # (2, 6)
print(s)

In [ ]:
# stack — adds a NEW axis
t = np.stack([p, q], axis=0)
print("stack axis=0 :", t.shape)    # (2, 2, 3) — two (2,3) arrays into a new dim

u = np.stack([p, q], axis=2)
print("stack axis=2 :", u.shape)    # (2, 3, 2)

A common image use-case: build an RGB image from three single-channel arrays.

```python
rgb = np.stack([r_channel, g_channel, b_channel], axis=2)  # (H, W, 3)
```

### Exercises

**Exercise 0.1:** Create a 1-D array of integers from 0 to 255 (use `np.arange`), then reshape it to `(16, 16)` and print its shape.
Next, slice out the **center 8×8 block** (rows 4–11, cols 4–11) and compute its mean with `.mean()`.

In [ ]:
# YOUR CODE HERE

**Exercise 0.2:** Given `a = np.arange(4)` (shape `(4,)`) and `b = np.arange(3)` (shape `(3,)`),
compute the **outer product** — a `(4, 3)` matrix where element `[i, j] = a[i] * b[j]`.

*Hint:* Use `None` indexing to make their shapes broadcastable: reshape `a` to `(4, 1)` and keep `b` as `(3,)`, then multiply.

In [ ]:
# YOUR CODE HERE

---
## 1. Loading and Exploring Images

We use **PIL** (`Pillow`) to open image files and immediately convert them to NumPy arrays. This gives us full control through array indexing and arithmetic.

Key functions:
- `Image.open(path)` — returns a PIL Image object
- `np.array(pil_img)` — converts it to a NumPy ndarray
- `.shape`, `.dtype`, `.min()`, `.max()` — basic array inspection

In [ ]:
img_color = np.array(Image.open(IMAGE_PATH))

print(f"Shape : {img_color.shape}")   # (H, W, C)
print(f"Dtype : {img_color.dtype}")   # uint8
print(f"Min   : {img_color.min()}")
print(f"Max   : {img_color.max()}")

In [ ]:
show_images(img_color, titles=["Color image (RGB)"])

### Creating and saving a grayscale version

`Image.convert('L')` uses the ITU-R 601-2 luma formula:

$$L = 0.299 R + 0.587 G + 0.114 B$$

We save it to disk so we can load it directly in later sections.

In [ ]:
img_gray = np.array(Image.open(IMAGE_PATH).convert("L"))

print(f"Shape : {img_gray.shape}")   # (H, W) — no channel dimension
print(f"Dtype : {img_gray.dtype}")

# Save for later use
Image.fromarray(img_gray).save(GRAY_PATH)
print(f"Saved grayscale image to {GRAY_PATH}")

In [ ]:
show_images(img_gray, titles=["Grayscale image"])

### Accessing individual pixels

- Color pixel → a 1D array of 3 values `[R, G, B]`
- Grayscale pixel → a single scalar value

In [ ]:
row, col = 100, 200

print(f"Color pixel at ({row}, {col}): {img_color[row, col]}")
print(f"Gray  pixel at ({row}, {col}): {img_gray[row, col]}")

---
## 2. Channel Extraction

An RGB image has 3 channels stored along axis 2. We can slice them out individually.

When displaying a single channel with `imshow`, use `cmap='gray'` — otherwise matplotlib applies a colormap that can be misleading.

In [ ]:
r_channel = img_color[:, :, 0]
g_channel = img_color[:, :, 1]
b_channel = img_color[:, :, 2]

show_images(img_color, r_channel, g_channel, b_channel,
            titles=["Original", "Red channel", "Green channel", "Blue channel"])

### Exercises

**Exercise 2.1:** Create three *color* images that isolate each channel:
- **Red-only**: keep the R channel, set G and B to 0
- **Green-only**: keep G, set R and B to 0
- **Blue-only**: keep B, set R and G to 0

Display all four (original + 3 filtered) in a row using `show_images`.

*Hint:* Start with `np.zeros_like(img_color)` to get a black image of the same shape, then copy one channel into it: `red_only[:, :, 0] = img_color[:, :, 0]`.

In [ ]:
# YOUR CODE HERE

---
## 3. Flip, Rotate, and Crop

NumPy slicing gives us geometric transforms for free:

| Operation | Code |
|---|---|
| Vertical flip | `img[::-1]` |
| Horizontal flip | `img[:, ::-1]` |
| 90° rotation | `np.rot90(img)` |
| Crop | `img[y1:y2, x1:x2]` |

In [ ]:
show_images(img_color, img_color[:, ::-1], img_color[::-1], np.rot90(img_color),
            titles=["Original", "H-flip", "V-flip", "Rot 90°"])

### Cropping

Remember the coordinate order: `img[row_start:row_end, col_start:col_end]`, i.e., **y first, x second**.

In [ ]:
H, W = img_color.shape[:2]
cropped = img_color[H//4 : 3*H//4, W//4 : 3*W//4]

show_images(img_color, cropped,
            titles=[f"Original ({H}×{W})", f"Center crop ({cropped.shape[0]}×{cropped.shape[1]})"])

### Exercises

**Exercise 3.1:** Rotate the image by 180° using **only slicing** (do not use `np.rot90`).

*Hint:* A 180° rotation is the same as flipping both vertically and horizontally.

In [ ]:
# YOUR CODE HERE

**Exercise 3.2:** Crop a region of your choice, then create a 1×2 figure showing the crop and its horizontally-flipped version.

In [ ]:
# YOUR CODE HERE

---
## 4. Point Processing: Arithmetic Operations

Point processing transforms each pixel **independently**: $g(x,y) = T[f(x,y)]$.

The simplest transforms are arithmetic: add, subtract, multiply.

> **Critical:** Images are stored as `uint8` (0–255). Arithmetic can overflow or underflow. Always **cast to a wider type first**, then **clip** back to [0, 255].

In [ ]:
gray = np.array(Image.open(GRAY_PATH))
print(f"Shape: {gray.shape}, dtype: {gray.dtype}")

### Brightness: Add / Subtract

In [ ]:
bright = np.clip(gray.astype(np.int16) + 80, 0, 255).astype(np.uint8)
dark   = np.clip(gray.astype(np.int16) - 80, 0, 255).astype(np.uint8)

show_images(dark, gray, bright, titles=["Darker (−80)", "Original", "Brighter (+80)"])

### Contrast: Multiply

In [ ]:
high_contrast = np.clip(gray.astype(np.float64) * 2.0, 0, 255).astype(np.uint8)
low_contrast  = (gray.astype(np.float64) * 0.5).astype(np.uint8)

show_images(low_contrast, gray, high_contrast,
            titles=["×0.5 (low contrast)", "Original", "×2.0 (high contrast)"])

### Complement / Negative

In [ ]:
negative = 255 - gray  # uint8 subtraction is safe here since 255 >= any pixel

show_images(gray, negative, titles=["Original", "Negative (255 − f)"])

### Why casting matters: uint8 overflow demo

`uint8` arithmetic **wraps around** modulo 256. For example, `200 + 100 = 44` (not 300). This produces garbage results.

In [ ]:
bad_result  = gray + np.uint8(200)                                       # WRONG: uint8 overflow
good_result = np.clip(gray.astype(np.int16) + 200, 0, 255).astype(np.uint8)  # CORRECT

show_images(gray, bad_result, good_result,
            titles=["Original", "uint8 overflow (WRONG)", "With clipping (CORRECT)"])

### Visualizing transfer functions

A transfer function maps each input intensity to an output intensity. We can plot them on a 256×256 grid.

In [ ]:
x = np.arange(256)

plt.figure(figsize=(5, 5))
plt.plot(x, x, "k--", label="Identity")
plt.plot(x, np.clip(x + 80, 0, 255), label="+80")
plt.plot(x, np.clip(x - 80, 0, 255), label="−80")
plt.plot(x, np.clip(x * 2, 0, 255), label="×2")
plt.plot(x, (x * 0.5).astype(int), label="×0.5")
plt.xlabel("Input intensity")
plt.ylabel("Output intensity")
plt.title("Transfer functions")
plt.legend()
plt.tight_layout()
plt.show()

### Exercises

**Exercise 4.1:** Create a "high-contrast negative" of the grayscale image in two steps:
1. Invert it: `negative = 255 - gray`
2. Multiply by 1.5 to boost contrast (don't forget to cast, clip, and cast back)

Display the original and the result side by side.

In [ ]:
# YOUR CODE HERE

**Exercise 4.2:** Apply brightness adjustment (+60) to the **color** image (all three channels). Remember to cast to `int16` before adding, then clip back to `[0, 255]` and convert to `uint8`. Display before and after with `show_images`.

In [ ]:
# YOUR CODE HERE

---
## 5. Gamma Correction

Gamma correction applies a **power-law** transform:

$$g = 255 \left(\frac{f}{255}\right)^\gamma$$

- $\gamma < 1$: brightens dark regions (expands shadows)
- $\gamma = 1$: identity (no change)
- $\gamma > 1$: darkens the image (compresses shadows)

We normalize to [0, 1], apply the power, then scale back to [0, 255].

In [ ]:
x = np.arange(256)

plt.figure(figsize=(5, 5))
plt.plot(x, x, "k--", label="γ = 1.0 (identity)")
for gamma in [0.3, 0.5, 2.0, 3.0]:
    plt.plot(x, 255 * (x / 255.0) ** gamma, label=f"γ = {gamma}")
plt.xlabel("Input")
plt.ylabel("Output")
plt.title("Gamma curves")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def apply_gamma(img, gamma):
    """Apply gamma correction to a uint8 image (grayscale or color)."""
    return (255 * (img / 255.0) ** gamma).astype(np.uint8)

In [ ]:
show_images(apply_gamma(gray, 0.4), gray, apply_gamma(gray, 2.5),
            titles=["γ = 0.4 (brighten)", "Original", "γ = 2.5 (darken)"])

### Gamma on a color image

The same function works element-wise on all three channels.

In [ ]:
show_images(apply_gamma(img_color, 0.4), img_color, apply_gamma(img_color, 2.5),
            titles=["γ = 0.4", "Original", "γ = 2.5"])

### Exercises

**Exercise 5.1:** Apply 5 different gamma values (0.2, 0.5, 1.0, 2.0, 5.0) to the grayscale image using the `apply_gamma` function defined above. Display all 5 results in a single row with `show_images`, using the gamma value as each title (e.g., `"γ = 0.2"`).

In [ ]:
# YOUR CODE HERE

---
## 6. Lookup Tables (LUT)

Any point transform $T: [0,255] \to [0,255]$ can be stored as an array of 256 entries. Applying it to an image is then a single **fancy-indexing** operation:

```python
T = np.array([...])  # shape (256,), dtype uint8
result = T[img]       # each pixel used as an index
```

This is extremely fast because there are no per-pixel arithmetic operations at runtime — just table lookups.

### Threshold LUT

In [ ]:
T_thresh = np.zeros(256, dtype=np.uint8)
T_thresh[128:] = 255

show_images(gray, T_thresh[gray], titles=["Original", "Threshold at 128"])

### More LUT examples: Solarize & Posterize

- **Solarize**: Invert pixels above a threshold → creates an "X" shaped transfer function
- **Posterize** (4 levels): Quantize 256 grey levels into just 4 steps

In [ ]:
x = np.arange(256)

# Solarize: invert pixels above 128
T_solar = x.copy().astype(np.uint8)
T_solar[128:] = 255 - T_solar[128:]

# Posterize: 4 levels (0, 85, 170, 255)
T_poster = np.zeros(256, dtype=np.uint8)
T_poster[0:64]    = 0
T_poster[64:128]  = 85
T_poster[128:192] = 170
T_poster[192:256] = 255

In [ ]:
show_images(T_thresh[gray], T_solar[gray], T_poster[gray],
            titles=["Threshold", "Solarize", "Posterize (4 levels)"])

We can also visualize the transfer functions (LUTs) themselves as line plots.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
for ax, lut, name in zip(axes, [T_thresh, T_solar, T_poster],
                          ["Threshold", "Solarize", "Posterize"]):
    ax.plot(x, lut)
    ax.plot(x, x, "k--", alpha=0.3)
    ax.set_title(name)
    ax.set_xlabel("Input")
    ax.set_ylabel("Output")
plt.tight_layout()
plt.show()

### Gamma as a LUT

Any formula-based transform can also be pre-computed as a LUT. This is especially useful when applying the same transform to many images.

In [ ]:
gamma_val = 0.4
T_gamma = (255 * (np.arange(256) / 255.0) ** gamma_val).astype(np.uint8)

lut_result    = T_gamma[gray]
direct_result = apply_gamma(gray, gamma_val)

show_images(lut_result, direct_result, titles=["Gamma via LUT", "Gamma via formula"])

### Exercises

**Exercise 6.1:** Build a custom **piecewise LUT** (a 256-element array `T`) with these rules:
- Input 0–63 → Output 0 (everything dark stays black)
- Input 64–191 → Output linearly increases from 0 to 255
- Input 192–255 → Output 255 (everything bright stays white)

Then:
1. Plot the transfer function: `plt.plot(np.arange(256), T)`
2. Apply it to the grayscale image with `T[gray]` and display the result

*Hint:* `np.linspace(0, 255, 128).astype(np.uint8)` gives 128 evenly spaced values from 0 to 255 — use this for the middle section.

In [ ]:
# YOUR CODE HERE

**Exercise 6.2:** Create a "warm tone" effect on the **color** image by applying a different gamma LUT to each channel:
- **Red**: gamma 0.8 (slightly brighter → warmer)
- **Green**: identity (no change)
- **Blue**: gamma 1.3 (slightly darker → less cool)

Build each LUT as a 256-element array, then apply them channel by channel:
```python
result[:, :, 0] = T_red[img_color[:, :, 0]]
```
Display before and after with `show_images`.

In [ ]:
# YOUR CODE HERE

---
## 7. Histograms

A histogram counts how many pixels have each intensity value (0–255). It reveals the **contrast characteristics** of an image at a glance.

Key function: `np.histogram(img.ravel(), bins=256, range=[0, 256])`

- `.ravel()` flattens the image to 1D (required by `np.histogram`)
- `bins=256` gives one bin per grey level
- Returns `(hist, bin_edges)` — we typically ignore `bin_edges`

In [ ]:
hist, bins = np.histogram(gray.ravel(), 256, [0, 256])

plt.figure(figsize=(8, 3))
plt.fill_between(np.arange(256), hist, alpha=0.7)
plt.xlabel("Pixel intensity")
plt.ylabel("Count")
plt.title("Histogram of grayscale image")
plt.tight_layout()
plt.show()

### Histogram shapes tell a story

Let's create dark, bright, and low-contrast versions and compare their histograms using `show_with_hist`.

In [ ]:
dark_img   = np.clip(gray.astype(np.float64) * 0.4, 0, 255).astype(np.uint8)
bright_img = np.clip(gray.astype(np.int16) + 100, 0, 255).astype(np.uint8)
low_cont   = np.clip(gray.astype(np.float64) * 0.3 + 100, 0, 255).astype(np.uint8)

show_with_hist(dark_img, gray, bright_img, low_cont,
               titles=["Dark", "Original", "Bright", "Low contrast"])

### RGB histogram overlay

For color images, we can plot the histogram of each channel on the same axes.

In [ ]:
plt.figure(figsize=(8, 3))
for ch, color in zip(range(3), ["red", "green", "blue"]):
    h, _ = np.histogram(img_color[:, :, ch].ravel(), 256, [0, 256])
    plt.fill_between(np.arange(256), h, alpha=0.35, color=color, label=color.capitalize())
plt.xlabel("Pixel intensity")
plt.ylabel("Count")
plt.title("RGB histogram")
plt.legend()
plt.tight_layout()
plt.show()

### Exercises

**Exercise 7.1:** Compute the histogram of the grayscale image, then find and print:
1. **Mode** (most frequent pixel value): the index with the highest count in the histogram. Use `np.argmax(hist)`.
2. **90% intensity range**: find the intensity values at the 5th and 95th percentiles. This tells you the range that contains the middle 90% of pixels.

*Hint:* Compute the **CDF** (cumulative sum of the histogram, `hist.cumsum()`), then normalize it by dividing by the total pixel count. The 5th percentile is the first index where `cdf >= 0.05`, and the 95th percentile where `cdf >= 0.95`. Use `np.searchsorted(cdf_normalized, 0.05)` to find these indices.

In [ ]:
# YOUR CODE HERE

**Exercise 7.2:** Compute the **negative** (complement) of the grayscale image: `negative = 255 - gray`.
Then plot both histograms side by side (use two subplots or `show_with_hist`). Observe the relationship between the two histograms and write your finding as a comment.

*Hint:* If the original has many pixels at intensity 50, where would those pixels appear in the negative's histogram?

In [ ]:
# YOUR CODE HERE

---
## 8. Histogram Stretching

Stretching linearly maps the actual intensity range $[f_{\min}, f_{\max}]$ to the full range $[0, 255]$:

$$g = \frac{f - f_{\min}}{f_{\max} - f_{\min}} \times 255$$

This is effective when the image has a genuinely narrow intensity range.

In [ ]:
# Create a low-contrast image
low = np.clip(gray.astype(np.float64) * 0.4 + 80, 0, 255).astype(np.uint8)
print(f"Low-contrast range: [{low.min()}, {low.max()}]")

# Stretch
f_min, f_max = low.min(), low.max()
stretched = ((low.astype(np.float64) - f_min) / (f_max - f_min) * 255).astype(np.uint8)
print(f"Stretched range:    [{stretched.min()}, {stretched.max()}]")

In [ ]:
show_with_hist(low, stretched, titles=["Low contrast", "Stretched"])

### When stretching fails

If even one pixel is 0 and another is 255, then $f_{\min}=0, f_{\max}=255$, and the formula simplifies to $g = f$ — no change.

In [ ]:
# Inject outlier pixels into the low-contrast image
low_with_outliers = low.copy()
low_with_outliers[0, 0] = 0
low_with_outliers[0, 1] = 255

f_min2, f_max2 = low_with_outliers.min(), low_with_outliers.max()
stretched2 = ((low_with_outliers.astype(np.float64) - f_min2) / (f_max2 - f_min2) * 255).astype(np.uint8)

print(f"Range with outliers: [{f_min2}, {f_max2}]  → stretching has no effect")

show_with_hist(low_with_outliers, stretched2,
               titles=["With outliers", "After stretching (no change)"])

### Exercises

**Exercise 8.1:** Implement **percentile-based stretching** — a version of histogram stretching that is robust to outliers.

Instead of using the absolute min/max, use the **2nd and 98th percentiles** as boundaries:

```python
lo = np.percentile(img, 2)    # ignore the darkest 2%
hi = np.percentile(img, 98)   # ignore the brightest 2%
```

Then apply the same stretching formula, but **clip** values outside `[lo, hi]` before stretching:

$$g = \frac{\text{clip}(f, lo, hi) - lo}{hi - lo} \times 255$$

Apply it to `low_with_outliers` and compare with basic stretching using `show_with_hist`.

In [ ]:
# YOUR CODE HERE

---
## 9. Histogram Equalization

Equalization redistributes pixel intensities so the histogram becomes approximately **uniform**. Unlike stretching, it considers the full distribution — not just min/max.

**Algorithm:**
1. Compute histogram $h[k]$
2. Compute CDF: $\text{CDF}[k] = \sum_{j=0}^{k} h[j]$
3. Normalize: $T[k] = \text{round}\left(\frac{\text{CDF}[k] - \text{CDF}_{\min}}{N - \text{CDF}_{\min}} \times 255\right)$
4. Apply as LUT: `result = T[img]`

In [ ]:
# Step-by-step equalization on the low-contrast image
hist_low, _ = np.histogram(low.ravel(), 256, [0, 256])

cdf = hist_low.cumsum()
cdf_normalized = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
cdf_normalized = cdf_normalized.astype(np.uint8)

equalized = cdf_normalized[low]

### Visualizing the CDF as a transfer function

The normalized CDF **is** the LUT used for equalization.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Histogram + CDF on twin axes
ax1 = axes[0]
ax2 = ax1.twinx()
ax1.fill_between(np.arange(256), hist_low, alpha=0.5, label="Histogram")
ax2.plot(np.arange(256), cdf / cdf.max(), "r-", label="CDF")
ax1.set_xlabel("Pixel intensity")
ax1.set_ylabel("Count")
ax2.set_ylabel("CDF (normalized)")
ax1.set_title("Histogram and CDF")
ax1.legend(loc="upper left")
ax2.legend(loc="center right")

# Transfer function (the LUT)
axes[1].plot(np.arange(256), cdf_normalized, label="Equalization LUT")
axes[1].plot(np.arange(256), np.arange(256), "k--", alpha=0.3, label="Identity")
axes[1].set_xlabel("Input")
axes[1].set_ylabel("Output")
axes[1].set_title("Transfer function")
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
show_with_hist(low, equalized, titles=["Low contrast", "Equalized"])

### Stretching vs Equalization

Let's compare both methods side by side on the same low-contrast input.

In [ ]:
show_images(low, stretched, equalized,
            titles=["Low contrast", "Stretched", "Equalized"])

### Exercises

**Exercise 9.1:** Implement histogram equalization from scratch as a function `equalize(img)`.

Steps (same as above):
1. Compute the histogram: `np.histogram(img.ravel(), 256, [0, 256])`
2. Compute the CDF: `hist.cumsum()`
3. Normalize: `(cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())`
4. Cast to `uint8` and apply as LUT: `T[img]`

Test it on the **original grayscale parrot** (not the artificial low-contrast version). Display original and equalized with their histograms using `show_with_hist`.

In [ ]:
# YOUR CODE HERE

---
## 10. HSV Color Space

**HSV** (Hue, Saturation, Value) separates *color* from *intensity*, which is often more intuitive than RGB.

We use `matplotlib.colors.rgb_to_hsv` and `hsv_to_rgb`:
- Input/output are **float arrays in [0, 1]** (so divide by 255 first, multiply by 255 after)
- Hue is in [0, 1] where 0 = red, 1/3 = green, 2/3 = blue, 1 = red again

In [ ]:
hsv = rgb_to_hsv(img_color / 255.0)

h_ch = hsv[:, :, 0]  # Hue        [0, 1]
s_ch = hsv[:, :, 1]  # Saturation [0, 1]
v_ch = hsv[:, :, 2]  # Value      [0, 1]

# Hue needs the 'hsv' colormap; S and V are shown as grayscale
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(img_color);              axes[0].set_title("Original")
axes[1].imshow(h_ch, cmap="hsv");       axes[1].set_title("Hue")
axes[2].imshow(s_ch, cmap="gray");      axes[2].set_title("Saturation")
axes[3].imshow(v_ch, cmap="gray");      axes[3].set_title("Value")
for ax in axes:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Hue shift

Shifting hue rotates all colors around the color wheel. Since hue is cyclic, we use modulo 1.0.

In [ ]:
hsv_shifted = hsv.copy()
hsv_shifted[:, :, 0] = (hsv_shifted[:, :, 0] + 1/3) % 1.0  # +120 degrees
shifted_rgb = (hsv_to_rgb(hsv_shifted) * 255).astype(np.uint8)

show_images(img_color, shifted_rgb, titles=["Original", "Hue +120°"])

### Saturation manipulation

In [ ]:
def adjust_saturation(img_rgb, factor):
    """Multiply saturation by factor, clip to [0, 1]."""
    hsv_tmp = rgb_to_hsv(img_rgb / 255.0)
    hsv_tmp[:, :, 1] = np.clip(hsv_tmp[:, :, 1] * factor, 0, 1)
    return (hsv_to_rgb(hsv_tmp) * 255).astype(np.uint8)

show_images(adjust_saturation(img_color, 0.3), img_color, adjust_saturation(img_color, 2.0),
            titles=["Desaturated (×0.3)", "Original", "Boosted (×2.0)"])

### Value (brightness) manipulation

In [ ]:
def adjust_value(img_rgb, factor):
    """Multiply Value channel by factor, clip to [0, 1]."""
    hsv_tmp = rgb_to_hsv(img_rgb / 255.0)
    hsv_tmp[:, :, 2] = np.clip(hsv_tmp[:, :, 2] * factor, 0, 1)
    return (hsv_to_rgb(hsv_tmp) * 255).astype(np.uint8)

show_images(adjust_value(img_color, 0.4), img_color, adjust_value(img_color, 1.5),
            titles=["Darker (V×0.4)", "Original", "Lighter (V×1.5)"])

### Practical application: Equalize only the V channel

Applying histogram equalization to each RGB channel independently can produce color artifacts. A better approach: convert to HSV, equalize only the Value channel (brightness), and convert back. This preserves the original colors.

In [ ]:
hsv_eq = rgb_to_hsv(img_color / 255.0)

v_uint8 = (hsv_eq[:, :, 2] * 255).astype(np.uint8)
hist_v, _ = np.histogram(v_uint8.ravel(), 256, [0, 256])
cdf_v = hist_v.cumsum()
cdf_v_norm = (cdf_v - cdf_v.min()) * 255 / (cdf_v.max() - cdf_v.min())
v_equalized = cdf_v_norm[v_uint8].astype(np.uint8)

hsv_eq[:, :, 2] = v_equalized / 255.0
result_v_eq = (hsv_to_rgb(hsv_eq) * 255).astype(np.uint8)

show_images(img_color, result_v_eq, titles=["Original", "V-channel equalized"])

### Exercises

**Exercise 10.1:** Compare two ways of getting a "grayscale" image:
- **V channel**: `(hsv[:, :, 2] * 255).astype(np.uint8)` — takes the max of R, G, B
- **Luma** (`Image.convert('L')`): uses the weighted formula $0.299R + 0.587G + 0.114B$

Extract the V channel, and display it alongside `img_gray` (which we created via `convert('L')` in Section 1) using `show_images`. Do they look the same? Where do you see differences?

*Hint:* Look at brightly colored regions — V = max(R,G,B) tends to be higher than the weighted luma average.

In [ ]:
# YOUR CODE HERE

**Exercise 10.2:** Create a 2×3 grid showing the original image and 5 hue-shifted versions (+60°, +120°, +180°, +240°, +300°). Label each with its shift angle.

*Hint:* Hue is normalized to [0, 1], so +60° = +1/6, +120° = +2/6, etc. Use modulo to wrap: `(h + offset) % 1.0`.

In [ ]:
# YOUR CODE HERE